# DANAM Image Metadata Notebook
This notebook is used along with the scripts clean_json and write_csv to query and analyze DANAM's image metadata quickly.

Queries is done via Pandas Dataframe.


In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

from datetime import datetime
from IPython.display import Markdown, display

from scripts.clean_json import clean_json


### Read DANAM json export
Always replace with the latest export

In [2]:
# read DANAM json export
danam_export = "json\DANAM\Monument_2022-04-25_04-41-33.json"
danam_images = clean_json(danam_export)
danam_df = pd.DataFrame(danam_images)

## Queries
### Checking images per monument

In [3]:
headers = ['danam_caption', 'empty_column', 'filename_danam', 'filetype',
       'filename_danam_2', 'filetype_2', 'filename', 'mon_id',
       'classification', 'notes', 'heidoc', 'heidata', 'validCaption',
       'caption', 'date1', 'date2', 'date', 'date3', 'agent', 'role', 'agent2',
       'role2', 'copyright', 'source', 'class_code', 'agent3', 'date_scan',
       'license', 'url', 'rights_text', 'lastModified']

In [19]:
# query through metadata table based on mon id

mon_id = 'KIR4013'
print("Monument ID: {}".format(mon_id))
mon = danam_df.loc[(danam_df['mon_id']==mon_id) | (danam_df['mon_id']==mon_id.upper())]
print("Number of images in DANAM: {}".format(mon.shape[0]))

### special queries ###
mon = mon.loc[mon['filename'].str.contains("")]
#mon = mon.loc[mon['validCaption']==False]
print("Images that matched query: {}".format(mon.shape[0]))

#select_headers = ['validCaption','filename', 'danam_caption', 'caption', 'date', 'agent', 'classification', 'notes']
select_headers = ['filename', 'danam_caption', 'caption', 'date', 'agent', 'role', 'classification', 'notes', 'copyright', 'source', 'license', 'rights_text']

mon = mon[select_headers].sort_values('filename')
mon[select_headers].sort_values('filename')

Monument ID: KIR4013
Number of images in DANAM: 24
Images that matched query: 24


,filename,danam_caption,caption,date,agent,role,classification,notes,copyright,source,license,rights_text
11697,KIR4013_D_2021_floor_plan,Cilancva Dhvākhā Phalcā; floor plan by Thomas Schrom; 2021-10 ;,"Cilancva Dhvākhā Phalcā, floor plan",2021-10,1248182,draftsman,Architekturzeichnung,,Nepal Heritage Documentation Project,,CC BY-SA 4.0,Nepal Heritage Documentation Project
11681,KIR4013_D_2021_location_map,"Cilancva Dhvākhā Phalcā; location map by Reinhard Herdick; 1975; updated by Thomas Schrom, 2021-10 ;","Cilancva Dhvākhā Phalcā, location map",1975,1249903,draftsman,Karte,,Nepal Heritage Documentation Project,,CC BY-SA 4.0,Nepal Heritage Documentation Project
11701,KIR4013_D_2021_section,Cilancva Dhvākhā Phalcā; section drawing by Thomas Schrom; 2021-10 ;,"Cilancva Dhvākhā Phalcā, section drawing",2021-10,1248182,draftsman,Architekturzeichnung,,Nepal Heritage Documentation Project,,CC BY-SA 4.0,Nepal Heritage Documentation Project
11700,KIR4013_D_2021_site_plan,"Cilancva Dhvākhā Phalcā; site plan by Reinhard Herdick; 1975; updated by Thomas Schrom, 2021-10 ;","Cilancva Dhvākhā Phalcā, site plan",1975,1249903,draftsman,Architekturzeichnung,,Nepal Heritage Documentation Project,,CC BY-SA 4.0,Nepal Heritage Documentation Project
11687,KIR4013_P_2015_01,"Cilancva Dhvākhā Phalcā, early stage of the post-earthquake reconstruction, view from NW; photo by Bharat Maharjan; 2015","Cilancva Dhvākhā Phalcā, early stage of the post-earthquake reconstruction, view from NW",2015,1208644,photographer,Architekturfotografie,,Nepal Heritage Documentation Project,,CC BY-SA 4.0,Nepal Heritage Documentation Project
11678,KIR4013_P_2015_02,"Cilancva Dhvākhā Phalcā, reconstruction of the roof framing, view from NW; photo by Bharat Maharjan; 2015","Cilancva Dhvākhā Phalcā, reconstruction of the roof framing, view from NW",2015,1208644,photographer,Architekturfotografie,,Nepal Heritage Documentation Project,,CC BY-SA 4.0,Nepal Heritage Documentation Project
11684,KIR4013_P_20211021_01,"Cilancva Dhvākhā Phalcā, view from E; photo by Thomas Schrom; 2021-10-21","Cilancva Dhvākhā Phalcā, view from E",2021-10-21,1248182,photographer,Architekturfotografie,,Nepal Heritage Documentation Project,,CC BY-SA 4.0,Nepal Heritage Documentation Project
11685,KIR4013_P_20211021_02,"Cilancva Dhvākhā Phalcā, view from E; photo by Thomas Schrom; 2021-10-21","Cilancva Dhvākhā Phalcā, view from E",2021-10-21,1248182,photographer,Architekturfotografie,,Nepal Heritage Documentation Project,,CC BY-SA 4.0,Nepal Heritage Documentation Project
11686,KIR4013_P_20211021_03,"Cilancva Dhvākhā Phalcā, view from NW; photo by Thomas Schrom; 2021-10-21","Cilancva Dhvākhā Phalcā, view from NW",2021-10-21,1248182,photographer,Architekturfotografie,,Nepal Heritage Documentation Project,,CC BY-SA 4.0,Nepal Heritage Documentation Project
11679,KIR4013_P_20211021_04,"Cilancva Dhvākhā Phalcā, view from W; photo by Thomas Schrom; 2021-10-21","Cilancva Dhvākhā Phalcā, view from W",2021-10-21,1248182,photographer,Architekturfotografie,,Nepal Heritage Documentation Project,,CC BY-SA 4.0,Nepal Heritage Documentation Project
